In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
mpl.rc('font', family = 'Malgun Gothic')
import matplotlib.pyplot as plt
import datetime
import math

import plotly.graph_objects as go # 그래프 그릴 때 사용할 라이브러리입니다.
from plotly.subplots import make_subplots

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# colab이랑 drive랑 연결

Mounted at /content/drive


In [ ]:
#상장폐지된 주가 데이터까지 모두 받아오기 -> 편향을 제거하기 위해서입니다.
kospi = pd.read_excel('/content/drive/MyDrive/DART/수정주가_240312.xlsx', index_col = 0, sheet_name='KOSPI')
kosdaq = pd.read_excel('/content/drive/MyDrive/DART/수정주가_240312.xlsx', index_col = 0, sheet_name='KOSDAQ')
price_data = pd.concat([kospi, kosdaq], axis = 1) # 코스피 주가와 코스닥 수정주가를 합쳐줍니다.

In [ ]:
price_data = price_data.loc[:,~price_data.T.duplicated()]

In [ ]:
eps = pd.read_excel('/content/drive/MyDrive/DART/스크리닝_240312.xlsx', index_col = 0, sheet_name = 'EPS' )

In [ ]:
quick_ratio = pd.read_excel('/content/drive/MyDrive/DART/스크리닝_240312.xlsx', index_col = 0, sheet_name = '당좌비율')
cashflow = pd.read_excel('/content/drive/MyDrive/DART/스크리닝_240312.xlsx',  index_col = 0, sheet_name = '영업활동현금흐름')

In [ ]:
quick_ratio = quick_ratio.iloc[:,1:]
cashflow = cashflow.iloc[:,1:]

2. 스크리닝 하기

In [ ]:
# 리밸런싱 조건 설정해주기
# 리밸런싱 주기
rebalancing = 91 #일 단위

# 거래비용
tax = 0.0002 # 현재 거래비용은 0.02%입니다.

# 투자원금
cash = 100000000 # 모의투자 시작금액인 1억으로 시작하겠습니다.
money = cash

risk_free_rate = 0.0359 # 무위험 자산 수익률은 23년 4월 1일 기준 CD91일물 금리로 하겠습니다.

# 손절기준
loss_cut = 0.1 #매수 이후 10% 이상 가격이 떨어지면 매도하겠습니다.

day = 0
quarter = 0
df_list = '현금 주식총액 포트폴리오가치 일일수익률 총수익률'.split() # 백테스트 데이터프레임에 보여줄 항목들입니다. 이후에 더 추가할거에요.
backtest = pd.DataFrame(columns = df_list)

In [ ]:
inv = price_data.iloc[day:day+rebalancing,:] #전체 주가 데이터에서 투자기간동안의 데이터를 뽑아냅니다.
inv = inv.replace(np.NaN, 0) # 수정주가 데이터에 없는 값들은 0으로 처리해줍니다.
inv = inv.loc[:,inv.iloc[0,:] > 0 ] # 투자 시작 시점에 데이터가 0인 친구들은 다 날려줍니다.

inv

,A005930,A000660,A005380,A000270,A068270,A005490,A035420,A051910,A105560,A006400,...,A128910,A130960,A131390,A134780,A138690,A141070,A149940,A900010,A900040,A900060
Symbol,,,,,,,,,,,,,,,,,,,,,
2013-01-01,30440,25750,218500,56500,38268,349000,71256,330000,37900,151000,...,2000.0,26500.0,3145.0,1552.0,18487.0,2641.0,3450.0,865.0,1372.0,2500.0
2013-01-02,31520,26600,216000,56300,38708,360500,71413,340000,38800,152500,...,2000.0,27500.0,3240.0,1552.0,18960.0,2817.0,3570.0,866.0,1372.0,2590.0
2013-01-03,30860,26650,206000,54600,38122,370000,69373,342500,39300,155500,...,2000.0,27600.0,3260.0,1501.0,19210.0,2753.0,3560.0,872.0,1350.0,2605.0
2013-01-04,30500,26350,206000,53600,38562,367000,71099,331000,39150,157000,...,2000.0,27900.0,3220.0,1498.0,19794.0,2787.0,3650.0,872.0,1327.0,2590.0
2013-01-05,30500,26350,206000,53600,38562,367000,71099,331000,39150,157000,...,2000.0,27900.0,3220.0,1498.0,19794.0,2787.0,3650.0,872.0,1327.0,2590.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-03-28,30200,29250,223500,56100,39001,327000,84126,265000,37000,140500,...,2090.0,35300.0,4050.0,2092.0,35446.0,2723.0,3850.0,1195.0,1309.0,3140.0
2013-03-29,30540,28750,224000,55600,39515,326000,86951,268000,37500,140500,...,2100.0,34850.0,4020.0,2092.0,30163.0,2740.0,4015.0,1200.0,1300.0,3140.0
2013-03-30,30540,28750,224000,55600,39515,326000,86951,268000,37500,140500,...,2100.0,34850.0,4020.0,2092.0,30163.0,2740.0,4015.0,1200.0,1300.0,3140.0


In [ ]:
# 스크리닝
quick_ratio_inv = pd.DataFrame(quick_ratio.iloc[quarter, :]) # 투자 시점의 당좌비율 데이터를 불러와줍니다.
quick_ratio_inv = quick_ratio_inv.dropna(axis = 'index') # 투자 시점에 없는 데이터들은 제거해줍니다.
quick_ratio_inv = quick_ratio_inv.sort_values(by = quick_ratio_inv.columns[0], ascending = False) # 내림차순으로 정렬해주고
quick_ratio_inv = quick_ratio_inv[quick_ratio_inv[quick_ratio_inv.columns[0]] > 100] # 당좌비율이 100 이상인 기업들만 남깁니다.
quick_ratio_inv_list = quick_ratio_inv.index # 리스트로 저장해줍니다.

quick_ratio_inv_list

Index(['A084990', 'A038620', 'A009810', 'Item', 'A119860', 'A015540',
       'A063080', 'A060280', 'A067770', 'A092460',
       ...
       'A023530', 'A023810', 'A083790', 'A044780', 'A053350', 'A065560',
       'A038870', 'A090150', 'A007070', 'A043260'],
      dtype='object', length=614)

In [ ]:
cashflow_inv = pd.DataFrame(cashflow.iloc[quarter, :]) # 투자 시점의 영업활동 현금흐름 데이터를 불러와줍니다.
cashflow_inv.dropna(axis = 'index', inplace = True) # NaN값 제거해주고
cashflow_inv = cashflow_inv.sort_values(by = cashflow_inv.columns[0], ascending=False) # 내림차순으로 정렬하겠습니다.
cashflow_inv = cashflow_inv[cashflow_inv.iloc[:,0] > 0] # 영업활동 현금흐름이 0이상인 기업만 추려줍니다.
cashflow_inv_list = cashflow_inv.index #리스트로 저장해줍니다.

cashflow_inv_list

Index(['A005930', 'A015760', 'A000030', 'A086790', 'A003600', 'A032830',
       'A005380', 'A034220', 'A032710', 'A009540',
       ...
       'A044780', 'A900100', 'A011700', 'A039860', 'A004870', 'A067290',
       'A042520', 'A035620', 'Item', 'A028300'],
      dtype='object', length=842)

In [ ]:
eps_inv = eps.iloc[day : day+rebalancing, :] # 투자 기간동안의 EPS 데이터를 불러와줍니다
eps_inv = eps_inv.replace(np.NaN, 0) # NaN 값을 0으로 대체합니다.
eps_inv = eps_inv.loc[:,eps_inv.iloc[0,:] > 0 ] # eps가 0 초과인 기업들만 골라줍니다.

eps_inv

,A005930,A000660,A373220,A207940,A005380,A000270,A068270,A005490,A051910,A006400,...,A900080,A900090,A900130,A900150,A900180,A950010,A950030,A950070,A950100,A950180
Symbol,,,,,,,,,,,,,,,,,,,,,
2013-01-01,175282.02,4044.59,0.0,-11421.61,29921.1,9416.39,1090.42,15786.74,17130.82,2768.31,...,669.58,328.08,-177.94,0.0,557.46,-0.3,0.0,0.0,1342.80,0.0
2013-01-02,175282.02,4044.59,0.0,-11421.61,29921.1,9416.39,1090.42,15786.74,17130.82,2768.31,...,669.58,328.08,-177.94,0.0,557.46,-0.3,0.0,0.0,1342.80,0.0
2013-01-03,175282.02,4044.59,0.0,-11421.61,29921.1,9416.39,1090.42,15786.74,17130.82,2768.31,...,669.58,328.08,-177.94,0.0,557.46,-0.3,0.0,0.0,1342.80,0.0
2013-01-04,175282.02,4044.59,0.0,-11421.61,29921.1,9416.39,1090.42,15786.74,17130.82,2768.31,...,669.58,328.08,-177.94,0.0,557.46,-0.3,0.0,0.0,1342.80,0.0
2013-01-05,175282.02,4044.59,0.0,-11421.61,29921.1,9416.39,1090.42,15786.74,17130.82,2768.31,...,669.58,328.08,-177.94,0.0,557.46,-0.3,0.0,0.0,1342.80,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-03-28,175282.02,4044.59,0.0,-11421.61,29921.1,9416.39,1090.42,15786.74,17130.82,2768.31,...,669.58,328.08,-177.94,0.0,557.46,-0.3,0.0,0.0,1342.80,0.0
2013-03-29,175282.02,4044.59,0.0,-11421.61,29921.1,9416.39,1090.42,15786.74,17130.82,2768.31,...,669.58,328.08,-177.94,0.0,557.46,-0.3,0.0,0.0,1342.80,0.0
2013-03-30,175282.02,4044.59,0.0,-11421.61,29921.1,9416.39,1090.42,15786.74,17130.82,2768.31,...,669.58,328.08,-177.94,0.0,557.46,-0.3,0.0,0.0,1342.80,0.0


In [ ]:
constraint = quick_ratio_inv_list.intersection(cashflow_inv_list) # 두 조건 모두에 걸리는 종목들을 선정해줍니다
constraint = constraint.intersection(eps_inv.columns) # 다시 eps와 조건에 맞게 선정해줍니다

inv_list = eps_inv[constraint].iloc[0,:].sort_values(ascending = False)[:20].index # 투자 후보 기업들 20개 최종적으로 리스트로 정리합니다.

print('투자 후보 갯수는 : ', len(inv_list)) # len 함수는 개수를 세어줍니다.

#여기까지 스크리닝 과정입니다.

투자 후보 갯수는 :  20


In [ ]:
final_inv_list = []
for i in range(len(inv_list)):
    if inv_list[i] in inv.columns:
        final_inv_list.append(inv_list[i])
    else:
        print(inv_list[i],' 종목이 없습니다')
print('투자하는 종목의 수는 : ', len(final_inv_list))
print('투자종목 : ',final_inv_list)
# 스크리닝 된 종목 들 중 가격 데이터에 있는 종목들만 투자 리스트에 추가해줍니다.

투자하는 종목의 수는 :  20
투자종목 :  ['A005930', 'A003240', 'A035420', 'A107590', 'A090430', 'A001630', 'A004990', 'A012330', 'A007310', 'A023530', 'A010130', 'A002380', 'A028150', 'A002790', 'A051910', 'A011210', 'A004370', 'A017390', 'A003200', 'A003800']


In [ ]:
# 매수 기준 : 동일 비중
# 보유한 주식의 가치 평가하기

if len(final_inv_list)==0:
    allocation=0
else:
    allocation = money / len(final_inv_list) # 동일 비중이기 때문에 보유 현금 / 투자할 종목수 로 나눠줍니다.


print('동일 비중 투자 금액은 : ' , allocation) # 저는 제 코드를 믿지 못하기 때문에 이렇게 중간중간 어디서 에러가 나는지 예상할 수 있게 출력 결과도 함께 프린트합니다.

동일 비중 투자 금액은 :  5000000.0


In [ ]:
final_price_data = inv[final_inv_list].copy()
vec = pd.DataFrame({'매수수량' : allocation // final_price_data.iloc[0,:]}) # 종목당 매수할 수를 데이터프레임(벡터) 로 만들어줍니다.
vec = vec.replace(np.NaN, 0)
vec

,매수수량
A005930,164.0
A003240,5.0
A035420,70.0
A107590,189.0
A090430,41.0
A001630,192.0
A004990,79.0
A012330,17.0
A007310,22.0
A023530,14.0


In [ ]:
# 매도(손절 기준) : 10% 이상 떨어질 경우 매도

# 손절을 할 때 보유하고 있는 주식수를 0으로 만들거나
# 손절 이후 포트폴리오의 가격을 0으로 만들거나의 방법을 선택할 수 있겠죠?
# 저는 손절 한 순간부터 포트폴리오에서 주가를 계산하지 않아도 되는 방법이 더 편할 것 같아서 이렇게 선택했지만
# 저와 다르게 계산하셔도 상관없습니다.
loss_cut_money_list = []
loss_cut_money = 0
for days in final_price_data.index: # 다음날로 넘어가는 반복문입니다.
    for stocks in final_price_data.columns: # 다음 종목으로 넘어가는 반복문입니다.
        if final_price_data.loc[days, stocks] < final_price_data[stocks][0] * ( 1 - loss_cut ): # (days, stocks) 위치에 있는 가격과 첫 매수날짜의 가격*0.9 를 비교합니다. # 가격이 10%이상 떨어지는지 비교합니다.
            loss_cut_money = loss_cut_money + (final_price_data.loc[days, stocks] * float(vec.loc[stocks])) * (1 - tax) # 만약 10%이상 하락한 경우, 손절가격 + (손절한 날의 해당 종목 가격 * vec로 저장한 매수 수량 * 세금)
            final_price_data.loc[days:, stocks] = 0 # 손절 후 해당 주식을 팔았으니 그 순간부터 해당 종목의 주가는 0으로 대체하겠습니다.

    loss_cut_money_list.append(loss_cut_money) # 손절한 금액을 로스컷머니 리스트에 저장합니다.
loss_cut_money_list # 보이시죠?

In [ ]:
product = np.dot(final_price_data, vec) #np.dot 통해 행렬과 벡터의 내적값을 구해줍니다. 해당 종목의 가격과 매수 수량을 곱하기 위함입니다. 매수대금이 계산됩니다.
product = pd.DataFrame(product) #데이터프레임으로 만들어서 나중에 붙여주겠습니다.
product

In [ ]:
balance = pd.DataFrame(index = product.index) # 인덱스는 product 데이터프레임의 인덱스를 가져옵니다.
balance['현금'] = money - product.iloc[0,0] # 현금은 리밸런싱이 이루어지는 날의 현금에서 주식 매수대금을 뺀 값입니다.
balance['현금'] += loss_cut_money_list # 손절한 금액은 날짜별로 더해줍시다.
balance

In [ ]:
_backtest = pd.DataFrame(columns = df_list) # 빈 데이터프레임을 만들어줍니다.
_backtest['주식총액'] = product # product는 보유하고 있는 주식의 가치입니다.
_backtest['현금'] = balance['현금'] # 아까 계산한 일자별 현금 데이터를 넣어줍니다.

In [ ]:
_backtest['포트폴리오가치'] = _backtest['현금'] + _backtest['주식총액'] # 포트폴리오 가치는 현금과 주식총액의 합입니다.
_backtest['총수익률'] = (_backtest['포트폴리오가치']/cash) # 처음 들고 시작한 돈 대비 수익률
_backtest.index = price_data.index[day : day + rebalancing] # 인덱스는 매수기간부터 91일간의 날짜를 가져오겠습니다.
_backtest # 일일수익률은 마지막에 계산하여 추가하겠습니다.

In [ ]:
backtest = pd.concat([backtest, _backtest], axis = 0, ignore_index=False)
# 데이터프레임들을 계속 붙여주면서 백테스트 데이터를 만들겠습니다.
# 반복문을 돌면서 빈데이터프레임에 데이터를 채워가기 때문에 먼저 선언해준 백테스트 데이터프레임에 붙여줍니다.
backtest

In [ ]:
# 리밸런싱 이후 시작 금액은 주식 현금 + 주식총액 - 거래비용입니다.
# 손절 후 보유한 현금 + 마지막날에 주식을 모두 팔아야하니 주식총액*0.9를 해줍니다.
money = backtest.iloc[-1,0] + (backtest.iloc[-1,1] * (1 - tax))
day = day + rebalancing
quarter += 1

In [ ]:
print('전체 리밸런싱 횟수는 {}'.format(int(len(price_data)/rebalancing)))
for reb in range(int(len(price_data)/rebalancing)): #리밸런싱 횟수를 계산해줍니다. 전체 투자기간/리밸런싱 주기
    print( reb+1, '회 투자')
    print('현금 : ', money)
    inv = price_data.iloc[day:day+rebalancing,:] #전체 주가 데이터에서 투자기간동안의 데이터를 뽑아냅니다.
    inv = inv.replace(np.NaN, 0) # 수정주가 데이터에 없는 값들은 0으로 처리해줍니다.
    inv = inv.loc[:,inv.iloc[0,:] > 0 ] # 투자 시작 시점에 데이터가 0인 친구들은 다 날려줍니다.
    # 스크리닝


    quick_ratio_inv = pd.DataFrame(quick_ratio.iloc[quarter, :]) # 투자 시점의 데이터를 불러와줍니다.
    quick_ratio_inv = quick_ratio_inv.dropna(axis = 'index') # 투자 시점에 없는 데이터들은 제거해줍니다.
    quick_ratio_inv = quick_ratio_inv.sort_values(by = quick_ratio_inv.columns[0], ascending = False) # 내림차순으로 정렬해주고
    quick_ratio_inv = quick_ratio_inv[quick_ratio_inv[quick_ratio_inv.columns[0]] > 100] # 당좌비율이 100 이상인 기업들만 남깁니다.
    quick_ratio_inv_list = quick_ratio_inv.index # 리스트로 저장해줍니다.



    cashflow_inv = pd.DataFrame(cashflow.iloc[quarter, :])
    cashflow_inv.dropna(axis = 'index', inplace = True) # NaN값 제거해주고
    cashflow_inv = cashflow_inv.sort_values(by = cashflow_inv.columns[0], ascending=False) # 내림차순으로 정렬하겠습니다.
    cashflow_inv = cashflow_inv[cashflow_inv.iloc[:,0] > 0] # 영업활동 현금흐름이 0이상인 기업만 추려줍니다.
    cashflow_inv_list = cashflow_inv.index #리스트로 저장해줍니다.


    eps_inv = eps.iloc[day : day+rebalancing, :] # 투자 기간동안의 데이터를 불러와줍니다
    eps_inv = eps_inv.replace(np.NaN, 0)
    eps_inv = eps_inv.loc[:,eps_inv.iloc[0,:] > 0 ] # eps가 0 이상인 기업들만 골라줍니다.

    constraint = quick_ratio_inv_list.intersection(cashflow_inv_list) # 두 조건 모두에 걸리는 종목들을 선정해줍니다
    constraint = constraint.intersection(eps_inv.columns) # 다시 eps와 조건에 맞게 선정해줍니다

    inv_list = eps_inv[constraint].iloc[0,:].sort_values(ascending = False)[:20].index

    print('투자 후보 갯수는 : ', len(inv_list))

    #여기까지 스크리닝 과정입니다.

    final_inv_list = []
    for i in range(len(inv_list)):
        if inv_list[i] in inv.columns:
            final_inv_list.append(inv_list[i])
        else:
            print(inv_list[i],' 종목이 없습니다')
    print('투자하는 종목의 수는 : ', len(final_inv_list))
    print('투자종목 : ',final_inv_list)
    # 스크리닝 된 종목 들 중 가격 데이터에 있는 종목들만 투자 리스트에 추가해줍니다.


    # 매수 기준 : 동일 비중
    # 보유한 주식의 가치 평가하기

    if len(final_inv_list)==0:
        allocation=0
    else:
        allocation = money / len(final_inv_list) # 동일 비중이기 때문에 보유 현금 / 투자할 종목 수 로 나눠줍니다.


    print('동일 비중 투자 금액은 : ' , allocation) # 저는 제 코드를 믿지 못하기 때문에 이렇게 중간중간 어디서 에러가 나는지 예상할 수 있게 출력 결과도 함께 프린트합니다.
    final_price_data = inv[final_inv_list].copy()



    vec = pd.DataFrame({'매수수량' : allocation // final_price_data.iloc[0,:]}) # 종목당 금액을 가격으로 나누어 매수할 수를 데이터프레임(벡터) 로 만들어줍니다.
    vec = vec.replace(np.NaN, 0)

    #매도(손절 기준) : 10% 이상 떨어질 경우 매도

    # 손절을 할 때 보유하고 있는 주식수를 0으로 만들거나
    # 손절 이후 포트폴리오의 가격을 0으로 만들거나의 방법을 선택할 수 있겠죠?
    # 저는 손절 한 순간부터 포트폴리오에서 주가를 계산하지 않아도 되는 방법이 더 편할 것 같아서 이렇게 선택했지만
    # 저와 다르게 계산하셔도 상관없습니다.
    loss_cut_money_list = []
    loss_cut_money = 0
    for days in final_price_data.index:
        for stocks in final_price_data.columns:
            if final_price_data.loc[days, stocks] < final_price_data[stocks][0] * ( 1 - loss_cut ):
                loss_cut_money = loss_cut_money + (final_price_data.loc[days, stocks] * float(vec.loc[stocks])) * (1 - tax)
                final_price_data.loc[days:, stocks] = 0

        loss_cut_money_list.append(loss_cut_money)


        # 날짜에 저장된 값을 리스트에 추가해줍니다. 이 반복문을 돌면서 포트폴리오 투자기간동안의 현금이 저장될 것입니다.



    product = np.dot(final_price_data, vec) #np.dot 통해 행렬과 벡터의 내적값을 구해줍니다.
    product = pd.DataFrame(product) #데이터프레임으로 만들어서 나중에 붙여주겠습니다.



    balance = pd.DataFrame(index = product.index)
    balance['현금'] = money - product.iloc[0,0] # 현금은 리밸런싱이 이루어지는 날의 현금에서 주식 매수대금을 뺀 값입니다.
    balance['현금'] += loss_cut_money_list # 손절한 금액은 날짜별로 더해줍시다


    _backtest = pd.DataFrame(columns = df_list) # 빈 데이터프레임을 만들어줍니다.
    _backtest['주식총액'] = product # 보유하고 있는 주식의 가치입니다.
    _backtest['현금'] = balance['현금'] # 아까 계산한 일자별 현금 데이터를 넣어줍니다.


    _backtest['포트폴리오가치'] = _backtest['현금'] + _backtest['주식총액'] # 포트폴리오 가치는 현금과 주식총액의 합입니다.
    _backtest['총수익률'] = (_backtest['포트폴리오가치']/cash) # 처음 들고 시작한 돈 대비 수익률
    _backtest.index = price_data.index[day : day + rebalancing]


    backtest = pd.concat([backtest, _backtest], axis = 0, ignore_index=False)
    # 데이터프레임들을 계속 붙여주면서 백테스트 데이터를 만들겠습니다.
    # 반복문을 돌면서 빈데이터프레임에 데이터를 채워가기 때문에 먼저 선언해준 백테스트 데이터프레임에 붙여줍니다.


    # 리밸런싱 이후 시작 금액은 주식 판매대금 - 거래비용 + 현금입니다.
    money = backtest.iloc[-1,0] + (backtest.iloc[-1,1] * ( 1 - tax))
    day = day + rebalancing
    quarter += 1

backtest['일일수익률'] = backtest['포트폴리오가치'].pct_change()


In [ ]:
BM = pd.read_excel('/content/drive/MyDrive/DART/KOSPI_240312.xlsx',index_col=0) # 코스피 지수를 벤치마크로 설정
BM['일일수익률'] = BM['시가'].pct_change()
BM['총수익률'] = BM['시가']/BM['시가'][0]
BM

In [ ]:
# CAGR : 연평균 성장률 ( Compound Annual Growth Rate)
num_of_year = int(len(backtest)/365)

CAGR = (backtest['포트폴리오가치'][-1]/cash)**(1/num_of_year) - 1
print('포트폴리오 CAGR은 {} %'.format(CAGR*100))

BENCHMARK_CAGR = ((BM['시가'][3911] / BM['시가'][0])) ** (1 / num_of_year) - 1
print("벤치마크 CAGR은 {} %".format(BENCHMARK_CAGR*100))

In [ ]:
mdd = ((backtest['총수익률']/backtest['총수익률'].cummax())-1)
backtest['MDD'] = mdd

In [ ]:
mdd = ((BM['총수익률']/BM['총수익률'].cummax())-1)
BM['MDD'] = mdd

In [ ]:
# Sharpe Ratio = (자산 X의 기대수익률 – 무위험 자산 수익률) / 자산 X의 기대수익률의 표준편차
backtest_return = backtest['일일수익률'].mean()
backtest_std = backtest['일일수익률'].std()

BM_return = BM['일일수익률'].mean()
BM_std = BM['일일수익률'].std()

backtest_sharpe = (backtest_return - risk_free_rate) / backtest_std
BM_sharpe = (BM_return - risk_free_rate) / BM_std

print('BACKTEST 샤프 비율은 : ', backtest_sharpe)
print('BENCHMARK 샤프 비율은 : ', BM_sharpe)

In [ ]:
fig = make_subplots(rows=4, cols=1,
                    specs=[[{"rowspan":3}],
                          [None],
                          [None],
                          [{}]],
                   shared_xaxes=True,
                   vertical_spacing=0.2,
                   subplot_titles=("수익률","MDD"))

fig.add_trace(go.Scatter(name='코스피 총수익률', x=BM.index, y=BM['총수익률']),
             row=1, col=1)
fig.add_trace(go.Scatter(name='포트폴리오 총수익률', x=backtest.index, y=backtest['총수익률']),
             row=1, col=1)
fig.add_trace(go.Scatter(name='포트폴리오 MDD', x=backtest.index, y=backtest['MDD'], fill='tozeroy'),
             row=4, col=1)
fig.add_trace(go.Scatter(name='BENCHMARK MDD', x=BM.index, y=BM['MDD'], fill='tozeroy'),
             row=4, col=1)

fig.update_layout(height=800, width=1000, plot_bgcolor='rgb(240, 240,240)',
                 title_text="백테스트 결과")

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()